<a href="https://colab.research.google.com/github/YokoyamaLab/PythonBasics/blob/main/day04_q2FaceComponents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day04 課題Q2 アバターに使う顔のパーツを描こう

In [ ]:
#@title 学籍番号、名前
id = "" #@param {type:"string"}
name = "" #@param {type:"string"}

### 顔のパーツの実装

まずComponentクラスや各パーツの基本となるクラスをGoogle Driveからimportします。

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/Shareddrives/2023-35A01／情報科学 c/lib

from face_component import Component,Face,Eyes,Ears,Mouth,Nose,Hair,SmoothHair

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shareddrives/2023-35A01／情報科学 c/lib


では、講義で皆さんに割り当てた顔のパーツを2つ実装してください。

### 顔のパーツ①

指定された顔のパーツのウチ、最初のものを指定されたクラス名で定義してください。時間の許す限り凝った絵を追求しましょう。インスタンス変数のfeatureには、そのパーツを端的に表すタイトルを付けてください。

In [ ]:
# コードはここへ

### 顔のパーツ②

指定された顔のパーツのウチ、二番目のものを指定されたクラス名で定義してください。時間の許す限り凝った絵を追求しましょう。インスタンス変数のfeatureには、そのパーツを端的に表すタイトルを付けてください。

In [ ]:
# コードはここへ

### 検証用コード

ここで実装したパーツ以外は、講義ノートで作成したクラスを使って構いません。クラス定義はすでにインポートされていますので、クラス名を指定すれば使えます。

In [ ]:
# 検証用の描画コードはここへ

必ず検証用コードで自分の実装したコードを利用した顔が、想定通り描けている事を確認してから、提出してください。どんなものを実装したか確認しやすいので、顔を描画した状態で提出してください。